Built following the documentation from Mode's API (http://developer.modeanalytics.com/) and a Mandrill API tutorial (http://python.mfamt.org/content/recipes/apis-and-scraping/sending-a-mandrill-email/).

Future work needed to productionalize:
- Use Mode API to refresh the report before pulling it
- Use Mandrill API to include an attachment
- Set up a way for non-technical users to modify from/to/subject/report variables
- Put on a server
- Set up a cadence

In [1]:
import requests #for api calling
import json #for data manipulation
import os #for calling api token/pass
import mandrill #for sending email

In [2]:
'''
    Report params
'''

account = 'snapdocs' #2797
report = 'd656f78eb891' #random report I pulled : https://modeanalytics.com/snapdocs/reports/d656f78eb891
report_url = 'https://modeanalytics.com/api/{a}/reports/{r}'.format(a=account, r=report)
#note: the report must be freshly run, the image will only be accessible shortly thereafter
#there is a way to use the API to refresh the report, then listen for when it's done

mode_token = os.environ['MODE_TOKEN']
mode_token_password = os.environ['MODE_TOKEN_PASSWORD']

mandrill_api_key = os.environ['MANDRILL_API_KEY']

In [13]:
'''
    Make Mode API call
'''
response = requests.get(report_url, auth=(mode_token, mode_token_password))
rj = response.json()

In [14]:
'''
    Images to show in the email
'''
graph_img = rj['web_preview_image']
logo_img = 'https://snapdocs-storage.s3.amazonaws.com/uploads/company/logo/11/thumb_unnamed.png'

In [15]:
'''
    Set up Mandrill connection
'''
mandrill_client = mandrill.Mandrill(mandrill_api_key)

In [16]:
'''
    Message meta-details
'''
from_email = 'scott@snapdocs.com'
from_name = 'Scott/Data lead'
to_email = 'scott@breuds.com'
to_name = 'Scott @ Breuds Title Agency'
subject = 'Your weekly report from Snapdocs'

In [17]:
'''
    Compose message
'''
message = {
	'from_email': from_email,
	'from_name': from_name,
	'to': [{
		'email': to_email,
		'name': to_name,
		'type': 'to'
		}],
	'subject': subject,
    'html': '<img src={l} alt="logo"/> <p/> <img src={g} alt="My image" />'.format(l=logo_img, g=graph_img)
	}

In [18]:
'''
    Send email
'''
result = mandrill_client.messages.send(message=message)